In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np
cols = ['status', 'duration', 'credit_hist', 'purpose', 'credit_amt', 'savings', 'employment', 'installment_rate', 'personal_status', 'debtors', 'residencesince', 'property', 'age', 'install_plans', 'housing', 'existing_credits', 'job', 'maintenance_paying_people', 'telephone', 'foreign_worker', 'result']
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data'
df = pd.read_csv(url, delimiter=' ', names=cols, index_col=False)

In [2]:
df.head()

,status,duration,credit_hist,purpose,credit_amt,savings,employment,installment_rate,personal_status,debtors,...,property,age,install_plans,housing,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


In [3]:
def preprocess_german(df):
    df['status'] = df['status'].map({'A11': 0, 'A12': 1, 'A13': 2, 'A14': 3}).astype(int)
    df['credit_hist'] = df['credit_hist'].map({'A34': 0, 'A33': 1, 'A32': 2, 'A31': 3, 'A30': 4}).astype(int)
    df.loc[(df['credit_amt'] <= 2000), 'credit_amt'] = 0
    df.loc[(df['credit_amt'] > 2000) & (df['credit_amt'] <= 5000), 'credit_amt'] = 1
    df.loc[(df['credit_amt'] > 5000), 'credit_amt'] = 2
    df.loc[(df['duration'] <= 12), 'duration'] = 0
    df.loc[(df['duration'] > 12) & (df['duration'] <= 24), 'duration'] = 1
    df.loc[(df['duration'] > 24) & (df['duration'] <= 36), 'duration'] = 2
    df.loc[(df['duration'] > 36), 'duration'] = 3
    df['age'] = df['age'].apply(lambda x : 1 if x >= 45 else 0) # 1 if old, 0 if young
    df['savings'] = df['savings'].map({'A61': 0, 'A62': 1, 'A63': 2, 'A64': 3, 'A65': 4}).astype(int)
    df['employment'] = df['employment'].map({'A71': 0, 'A72': 1, 'A73': 2, 'A74': 3, 'A75': 4}).astype(int)
    df['gender'] = df['personal_status'].map({'A91': 1, 'A92': 0, 'A93': 1, 'A94': 1, 'A95': 0}).astype(int)
    df['debtors'] = df['debtors'].map({'A101': 0, 'A102': 1, 'A103': 2}).astype(int)
    df['property'] = df['property'].map({'A121': 3, 'A122': 2, 'A123': 1, 'A124': 0}).astype(int)
    df['install_plans'] = df['install_plans'].map({'A141': 1, 'A142': 1, 'A143': 0}).astype(int)
    df['job'] = df['job'].map({'A171': 0, 'A172': 1, 'A173': 2, 'A174': 3}).astype(int)
    df['telephone'] = df['telephone'].map({'A191': 0, 'A192': 1}).astype(int)
    df['foreign_worker'] = df['foreign_worker'].map({'A201': 1, 'A202': 0}).astype(int)
    pd.get_dummies(df, columns=['purpose', 'housing'], drop_first=True)

    return df

In [4]:
X = preprocess_german(df);

In [5]:
df.head()

,status,duration,credit_hist,purpose,credit_amt,savings,employment,installment_rate,personal_status,debtors,...,age,install_plans,housing,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result,gender
0,0,0,0,A43,0,4,4,4,A93,0,...,1,0,A152,2,2,1,1,1,1,1
1,1,3,2,A43,2,0,2,2,A92,0,...,0,0,A152,1,2,1,0,1,2,0
2,3,0,0,A46,1,0,3,2,A93,0,...,1,0,A152,1,1,2,0,1,1,1
3,0,3,2,A42,2,0,3,2,A93,2,...,1,0,A153,1,2,2,0,1,1,1
4,0,1,1,A40,1,0,2,3,A93,0,...,1,0,A153,2,2,2,0,1,2,1


In [6]:
df = df.drop(["purpose", "housing", "personal_status"], axis=1)

In [7]:
x = df[['status', 'duration', 'credit_hist', 'credit_amt', 'savings',
       'employment', 'installment_rate', 'debtors', 'residencesince',
       'property', 'age', 'install_plans', 'existing_credits', 'job',
       'maintenance_paying_people', 'telephone', 'foreign_worker',
       'gender']]

In [8]:
y = df[["result"]]

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=15)

In [21]:
def demographic_parity(x_train, y_train):
    # find the predictive probabilities for the sensitive and non-sensitive groups

    logistic_model = LogisticRegression(max_iter=1000)
    logistic_model.fit(x_train, y_train.values.ravel())
    y_pred = logistic_model.predict(x_test)
    y_pred_prob = logistic_model.predict_proba(x_test)
    accuracy_score(y_test, y_pred)
    groups_age =  df.groupby(df['age'])
    protected_group_age_df = groups_age.get_group(1)
    non_protected_group_age_df = groups_age.get_group(0)
    y_pred = logistic_model.predict(x_test)

    # There are various fairness metrics defined in the 'Fairness Definitions Explained' paper based on the predicted probabilities and actual outcome

    # 1. Test fairness -> P(Y=1|S=s, G=m) = P(Y=1|S=s, G=f), which can be used for age(protected and non-protected classes)
    # 2. Well-calibration -> P(Y=1|S=s, G=m) = P(Y=1|S=s, G=f) = s, which can be used for age(protected and non-protected classes)
    # 3. Balance for positive class -> E(S|Y=1,G=m) = E(S|Y=1,G=f), which can be used for age(protected and non-protected classes)
    # 4. Balance for negative class -> E(S|Y=0,G=m) = E(S|Y=0,G=f), which can be used for age(protected and non-protected classes)

    # Now, we need to find the P(ŷ = 1 | y = 1, G = 0) and P(ŷ = 1 | y = 1, G = 1) as G is the sensitive attribute here
    # Find the average y_pred_proba where the actual outcome Y = 1 for the divided groups

    non_protected_group_age_credresult_1_df = non_protected_group_age_df.groupby(non_protected_group_age_df['result']).get_group(1)

    #Drop the result column as the logistic regression model will accept 18 columns as input.
    non_protected_group_age_credresult_1_df = non_protected_group_age_credresult_1_df.drop(["result"], axis=1)

    # Need to find the predictive probability for the dataframe and then finally find the average
    y_pred_non_protected_group_age_credresult_1 = logistic_model.predict(non_protected_group_age_credresult_1_df)

    protected_group_age_credresult_1_df = protected_group_age_df.groupby(protected_group_age_df['result']).get_group(1)

    #Drop the result column as the logistic regression model will accept 18 columns as input.
    protected_group_age_credresult_1_df = protected_group_age_credresult_1_df.drop(["result"], axis=1)

    # Need to find the predict probability for the dataframe and then finally find the average
    y_pred_protected_group_age_credresult_1 = logistic_model.predict(protected_group_age_credresult_1_df)

    # Need to find the predict probability for the dataframe and then finally find the average
    y_pred_protected_group_age_credresult_1 = logistic_model.predict_proba(protected_group_age_credresult_1_df)
    y_pred_non_protected_group_age_credresult_1 = logistic_model.predict_proba(non_protected_group_age_credresult_1_df)

    protected_pp = y_pred_protected_group_age_credresult_1.transpose()[0].mean()
    non_protected_pp = y_pred_non_protected_group_age_credresult_1.transpose()[0].mean()

    # print(protected_pp)
    # print(non_protected_pp)

    return protected_pp - non_protected_pp

In [19]:
print(x_train.shape[0])

800


In [22]:
dp_ori = demographic_parity(x_train, y_train)
print(dp_ori)

0.05045275994726861


In [36]:
result = []

for i in range(len(x_train)):
    # x_train_new = x_train.append(x_train.iloc[i])
    x_train_new = pd.concat([x_train, x_train.iloc[i:i+1]])
    y_train_new = pd.concat([y_train, y_train.iloc[i:i+1]])
    # y_train_new = y_train.append(y_train.iloc[i])

    # print(x_train_new.shape[0])
    #print(demographic_parity(x_train_new, y_train_new))

    result.append((demographic_parity(x_train_new, y_train_new) - dp_ori, i))
    # print(demographic_parity(x_train_new, y_train_new) - dp_ori, i)


In [39]:
sorted_result = sorted(result, key=lambda x: x[0], reverse=True)

In [40]:
print(sorted_result)

[(0.0036118174140842285, 55), (0.003580731808373372, 171), (0.0035433836216983705, 310), (0.003229562694604504, 466), (0.0030206191655928194, 183), (0.0028856718534507886, 769), (0.002853375715542228, 688), (0.0027511332050284, 652), (0.0023681810100699696, 30), (0.002364690409308423, 5), (0.0023343717950049525, 487), (0.0022707615513531954, 276), (0.002231083671825118, 595), (0.002216005399080845, 716), (0.0021655473536447145, 354), (0.0021057266920427864, 628), (0.0020737698700615814, 291), (0.002008266316207541, 359), (0.001965341183862823, 633), (0.0019354724271974932, 517), (0.001935204488345943, 383), (0.0018813425090959957, 225), (0.001839423494033987, 718), (0.0017427226218683645, 701), (0.0016902609284954462, 201), (0.0016546217332784607, 410), (0.001533174751858657, 319), (0.001522349558828795, 83), (0.0015129606557447994, 95), (0.0015092965211398468, 213), (0.0015059689370894302, 720), (0.001449419600734747, 99), (0.001448077651556412, 13), (0.0014454013365660101, 629), (0.0